# Project Notebook: The Linear Regression Model

## 1. Introduction

We started by building intuition for model based learning, explored how the linear regression model worked, understood how the two different approaches to model fitting worked, and some techniques for cleaning, transforming, and selecting features. In this project, you can practice what you learned by exploring ways to improve the models we built.

You'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can also read about the different columns in the data [here](https://s3.amazonaws.com/dq-content/307/data_description.txt).

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models.

**Tasks**

1. Import pandas, matplotlib, and numpy into the environment. Import the classes you need from scikit-learn as well.
2. Read `AmesHousing.tsv` () into a pandas data frame.
3. For the following functions, we recommend creating them in the first few cells in the notebook. This way, you can add cells to the end of the notebook to do experiments and update the functions in these cells.
* Create a function named `transform_features()` that, for now, just returns the train data frame.
* Create a function named `select_features()` that, for now, just returns the Gr Liv Area and SalePrice columns from the train data frame.
* Create a function named `train_and_test()` that, for now:

1. Selects the first 1460 rows from from data and assign to train.
2. Selects the remaining rows from data and assign to test.
3. Trains a model using all numerical columns except the SalePrice column (the target column) from the data frame returned from `select_features()`
4. Tests the model on the test set and returns the `RMSE` value.

In [1]:
# Import pandas, matplotlib, and numpy into the environment
# Import the classes required from scikit-learn as well
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression




In [6]:
#read  and preview data
df= pd.read_csv("https://bit.ly/3boZCX4", sep='\t')
df.head()


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [7]:
# Creating a function that just returns the train data frame.
def transform_features(df):
  
  return df
  

In [8]:
# Creating a function that just returns the Gr Liv Area and SalePrice columns
# from the train data frame

def select_features(df):
  
  return df[['Gr Liv Area', 'SalePrice']]

In [9]:
#creating a function that fits, predicts and returns RMSE
def train_and_test(df):
  
  #splitting data into train and test 
  train = df[0:1460]
  test = df[1460:]
  
  #selecting numerical columns only
  numerical_train = train.select_dtypes(include = ['float','integer'])
  numerical_test = test.select_dtypes(include = ['float','integer'])
 
    
  features = numerical_train.columns.drop('SalePrice')
  target = 'SalePrice'

  #initializing model
  lr = LinearRegression()

  #fitting model
  lr.fit(train[features], train[target])

  #making predictions
  predicted_y = lr.predict(test[features])
   
  #computing MSE
  mse = mean_squared_error(test[target],predicted_y)

 #computing RMSE
  rmse = np.sqrt(mse)
 
  return rmse
   

In [10]:
#using the above defined functions to get RMSE
transform_data = transform_features(df)

features = select_features(transform_data)

rmse = train_and_test(features)
  
print(rmse)

57088.25161263909


## 2. Feature Engineering

Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. Update `transform_features()` so that any column from the data frame with more than 25% (or another cutoff value) missing values is dropped. You also need to remove any columns that leak information about the sale (e.g. like the year the sale happened). In general, the goal of this function is to:

* remove features that we don't want to use in the model, just based on the number of missing values or data leakage.
* transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc).
* create new features by combining other features.

Next, you need to get more familiar with the remaining columns by reading the data documentation for each column, determining what transformations are necessary (if any), and more. As we mentioned earlier, succeeding in predictive modeling (and competitions like Kaggle) is highly dependent on the quality of features the model has. Libraries like scikit-learn have made it quick and easy to simply try and tweak many different models, but cleaning, selecting, and transforming features are still more of an art that requires a bit of human ingenuity.

**Tasks**

1. As we mentioned earlier, we recommend adding some cells to explore and experiment with different features (before rewriting these functions).

2. The `transform_features()` function shouldn't modify the train data frame and instead return a new one entirely. This way, we can keep using train in the experimentation cells.

3. Which columns contain less than 5% missing values?
* For numerical columns that meet this criteria, let's fill in the missing values using the most popular value for that column.

4. What new features can we create, that better capture the information in some of the features?
* An example of this would be the `years_until_remod` feature we created in the last lesson.

5. Which columns need to be dropped for other reasons?
* Which columns aren't useful for machine learning?
* Which columns leak data about the final sale?

In [14]:
#find missing values per column
missing_values = df.isnull().sum().sort_values(ascending = False)

many_missing = missing_values[(missing_values/len(df) > .05)]
print(many_missing.sort_values(ascending=False))

#we shall drop these columns

df = df.drop(many_missing.index, axis = 1)

   


Series([], dtype: int64)


In [15]:
#finding and dropping text columns with missing values
text_col_missing = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

drop_missing = text_col_missing[text_col_missing > 0]

df = df.drop(drop_missing.index, axis=1)

In [16]:
 df.isnull().sum()

Order             0
PID               0
MS SubClass       0
MS Zoning         0
Lot Area          0
                 ..
Mo Sold           0
Yr Sold           0
Sale Type         0
Sale Condition    0
SalePrice         0
Length: 64, dtype: int64

In [19]:
#find numeric columns with missing columns
missing_numeric = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)

#5%
columns_to_fix = missing_numeric[(missing_numeric < len(df)/20) & (missing_numeric > 0)].sort_values(ascending = False)
columns_to_fix




Mas Vnr Area      23
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Area        1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
BsmtFin SF 1       1
Garage Cars        1
dtype: int64

In [21]:
#replacing missing values in numeric columns with mode

df['Mas Vnr Area'] = df['Mas Vnr Area'].fillna(df['Mas Vnr Area'].mode()[0])
df['Bsmt Full Bath'] = df['Bsmt Full Bath'].fillna(df['Bsmt Full Bath'].mode()[0])
df['Bsmt Half Bath'] = df['Bsmt Half Bath'].fillna(df['Bsmt Half Bath'].mode()[0])
df['Garage Area'] = df['Garage Area'].fillna(df['Garage Area'].mode()[0])
df['BsmtFin SF 2'] = df['BsmtFin SF 2'].fillna(df['BsmtFin SF 2'].mode()[0])
df['Bsmt Unf SF'] = df['Bsmt Unf SF'].fillna(df['Bsmt Unf SF'].mode()[0])
df['Total Bsmt SF'] = df['Total Bsmt SF'].fillna(df['Total Bsmt SF'].mode()[0])
df['BsmtFin SF 1'] = df['BsmtFin SF 1'].fillna(df['BsmtFin SF 1'].mode()[0])
df['Garage Cars'] = df['Garage Cars'].fillna(df['Garage Cars'].mode()[0])


#method 2 of replacing null values
repl_dict = df[columns_to_fix.index].mode().to_dict(orient='records')[0]
df = df.fillna(repl_dict)


In [23]:
#rechecking missing values

df.isnull().sum().value_counts()

0    64
dtype: int64

In [25]:
#creating more features

Remod_age = df['Yr Sold'] - df['Year Remod/Add']

#Adding to datafreame

df['Remod_age'] = Remod_age

In [26]:
#dropping columns not useful/columns that leak information to model


df = df.drop(["PID", "Order"], axis=1)

df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [29]:
#updating the def_transform function with all the above code

def transform_features(df):

  missing_values = df.isnull().sum().sort_values(ascending = False)
  many_missing = missing_values[(missing_values/len(df) > .05)]
  
  #we shall drop these columns

  df = df.drop(many_missing.index, axis = 1)

  #finding and dropping text columns with missing values
  text_col_missing = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

  drop_missing = text_col_missing[text_col_missing > 0]

  df = df.drop(drop_missing.index, axis=1)

  #find numeric columns with missing columns
  missing_numeric = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)

  #5%
  columns_to_fix = missing_numeric[(missing_numeric < len(df)/20) & (missing_numeric > 0)].sort_values(ascending = False)
  

  #replacing missing values in numeric columns with mode

  repl_dict = df[columns_to_fix.index].mode().to_dict(orient='records')[0]
  df = df.fillna(repl_dict)

  #creating more features

  Remod_age = df['Yr Sold'] - df['Year Remod/Add']

  #Adding to datafreame

  df['Remod_age'] = Remod_age

  #dropping columns not useful/columns that leak information to model


  df = df.drop(["PID", "Order"], axis=1)

  df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

  return df


def select_features(df):
  return df[['Gr Liv Area', 'SalePrice']]


def train_and_test(df):
  
  #splitting data into train and test 
  train = df[0:1460]
  test = df[1460:]
  
  #selecting numerical columns only
  numerical_train = train.select_dtypes(include = ['float','integer'])
  numerical_test = test.select_dtypes(include = ['float','integer'])
 
    
  features = numerical_train.columns.drop('SalePrice')
  target = 'SalePrice'

  #initializing model
  lr = LinearRegression()

  #fitting model
  lr.fit(train[features], train[target])

  #making predictions
  predicted_y = lr.predict(test[features])
   
  #computing MSE
  mse = mean_squared_error(test[target],predicted_y)

 #computing RMSE
  rmse = np.sqrt(mse)
 
  return rmse






In [49]:
#using the function above
df_housing= pd.read_csv("https://bit.ly/3boZCX4", sep='\t')

# Using above functions to process dataframe and return RMSE
transform_data = transform_features(df_housing)
features_df = select_features(transform_df)
rmse = train_and_test(features_df)

# Printing rmse from above
print(rmse)

57088.25161263909


## 3. Feature Selection

Now that we have cleaned and transformed a lot of the features in the data set, it's time to move on to feature selection for numerical features.

**Tasks**

1. Generate a correlation heatmap matrix of the numerical features in the training data set.
* Which features correlate strongly with our target column, `SalePrice`?
* Calculate the correlation coefficients for the columns that seem to correlate well with `SalePrice`. Because we have a pipeline in place, it's easy to try different features and see which features result in a better cross validation score.

2. Which columns in the data frame should be converted to the categorical data type? All of the columns that can be categorized as nominal variables are candidates for being converted to categorical. Here are some other things you should think about:
* If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
* Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).

3. Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

4. What are some ways we can explore which categorical columns "correlate" well with `SalePrice`?

5. Update the logic for the `select_features()` function. This function should take in the new, modified train and test data frames that were returned from `transform_features()`.

In [50]:
# numerical columns dataframe
num_df = transform_data.select_dtypes(include = ['integer', 'float'])

num_df.head()


,MS SubClass,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Remod_age
0,20,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,2.0,528.0,210,62,0,0,0,0,0,215000,50
1,20,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1.0,730.0,140,0,0,0,120,0,0,105000,49
2,20,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1.0,312.0,393,36,0,0,0,0,12500,172000,52
3,20,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,2.0,522.0,0,0,0,0,0,0,0,244000,42
4,60,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,2.0,482.0,212,34,0,0,0,0,0,189900,12


In [51]:
# checking correlation between feature columns and target column("SalesPrice")
target_corr = num_df.corr()['SalePrice'].abs().sort_values(ascending = False)
target_corr

SalePrice          1.000000
Overall Qual       0.799262
Gr Liv Area        0.706780
Garage Cars        0.647812
Garage Area        0.640138
Total Bsmt SF      0.632529
1st Flr SF         0.621676
Year Built         0.558426
Full Bath          0.545604
Remod_age          0.534940
Year Remod/Add     0.532974
Mas Vnr Area       0.502196
TotRms AbvGrd      0.495474
Fireplaces         0.474558
BsmtFin SF 1       0.433147
Wood Deck SF       0.327143
Open Porch SF      0.312951
Half Bath          0.285056
Bsmt Full Bath     0.275823
2nd Flr SF         0.269373
Lot Area           0.266549
Bsmt Unf SF        0.183308
Bedroom AbvGr      0.143913
Enclosed Porch     0.128787
Kitchen AbvGr      0.119814
Screen Porch       0.112151
Overall Cond       0.101697
MS SubClass        0.085092
Pool Area          0.068403
Low Qual Fin SF    0.037660
Bsmt Half Bath     0.035817
3Ssn Porch         0.032225
Misc Val           0.015691
BsmtFin SF 2       0.006018
Name: SalePrice, dtype: float64

In [52]:
#will select features with coefficients greater than 0.5 and drop those that do not meet

transform_data = transform_data.drop(target_corr[target_corr < 0.5].index, axis = 1)
transform_data

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Total Bsmt SF,Heating,Heating QC,Central Air,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,Functional,Garage Cars,Garage Area,Paved Drive,SalePrice,Remod_age
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,1960,1960,Hip,CompShg,BrkFace,Plywood,112.0,TA,TA,CBlock,1080.0,GasA,Fa,Y,1656,1656,1,TA,Typ,2.0,528.0,P,215000,50
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,1961,1961,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,CBlock,882.0,GasA,TA,Y,896,896,1,TA,Typ,1.0,730.0,Y,105000,49
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,108.0,TA,TA,CBlock,1329.0,GasA,TA,Y,1329,1329,1,Gd,Typ,1.0,312.0,Y,172000,52
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,1968,1968,Hip,CompShg,BrkFace,BrkFace,0.0,Gd,TA,CBlock,2110.0,GasA,Ex,Y,2110,2110,2,Ex,Typ,2.0,522.0,Y,244000,42
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,PConc,928.0,GasA,Gd,Y,928,1629,2,TA,Typ,2.0,482.0,Y,189900,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,1Fam,SLvl,6,1984,1984,Gable,CompShg,HdBoard,HdBoard,0.0,TA,TA,CBlock,1003.0,GasA,TA,Y,1003,1003,1,TA,Typ,2.0,588.0,Y,142500,22
2926,RL,Pave,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,1983,1983,Gable,CompShg,HdBoard,HdBoard,0.0,TA,TA,CBlock,864.0,GasA,TA,Y,902,902,1,TA,Typ,2.0,484.0,Y,131000,23
2927,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,0.0,TA,TA,PConc,912.0,GasA,TA,Y,970,970,1,TA,Typ,0.0,0.0,Y,132000,14
2928,RL,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,1974,1975,Gable,CompShg,HdBoard,HdBoard,0.0,TA,TA,CBlock,1389.0,GasA,Gd,Y,1389,1389,1,TA,Typ,2.0,418.0,Y,170000,31


In [53]:
# categorical column names list

categ_features = ["Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
"Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
"Misc Feature", "Sale Type", "Sale Condition"]

#list of categorical columns
transform_cols = []
for i in categ_features:
    if i in transform_df.columns:
        transform_cols.append(i)

#unique values in each 
unique = transform_df[transform_cols].apply(lambda i: len(i.value_counts())).sort_values(ascending = False)


drop_notunique_cols = unique[unique > 7].index


transform_data = transform_data.drop(drop_notunique_cols, axis=1)


# make rest of text columns to categorical
text_columns = transform_data.select_dtypes(include=['object'])
for i in text_columns:
    transform_data[i] = transform_data[i].astype('category')
    
transform_data = pd.concat([
    transform_data, 
    pd.get_dummies(transform_data.select_dtypes(include=['category']))
], axis=1)

## 4. Train and Test

Now for the final part of the pipeline, training and testing. When iterating on different features, using simple validation is a good idea. Let's add a parameter named `k` that controls the type of cross validation that occurs.

**Tasks**

1. The optional `k` parameter should accept integer values, with a default value of `0`.

2. When `k` equals `0`, perform holdout validation (what we already implemented):

* Select the first `1460` rows and assign to `train`.
* Select the remaining rows and assign to test.
* Train on `train` and `test` on test.
* Compute the `RMSE` and return.

3. When k equals 1, perform simple cross validation:

* Shuffle the ordering of the rows in the data frame.
* Select the first 1460 rows and assign to `fold_one`.
* Select the remaining rows and assign to `fold_two`.
* Train on `fold_one` and test on `fold_two`.
* Train on `fold_two` and test on `fold_one`.
* Compute the average RMSE and return.

4. When `k` is greater than `0`, implement k-fold cross validation using `k` folds:

* Perform `k-fold` cross validation using k folds.
* Calculate the average `RMSE` value and return this value.

In [56]:
def transform_features(df):

  missing_values = df.isnull().sum().sort_values(ascending = False)
  many_missing = missing_values[(missing_values/len(df) > .05)]
  
  #we shall drop these columns

  df = df.drop(many_missing.index, axis = 1)

  #finding and dropping text columns with missing values
  text_col_missing = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

  drop_missing = text_col_missing[text_col_missing > 0]

  df = df.drop(drop_missing.index, axis=1)

  #find numeric columns with missing columns
  missing_numeric = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)

  #5%
  columns_to_fix = missing_numeric[(missing_numeric < len(df)/20) & (missing_numeric > 0)].sort_values(ascending = False)
  

  #replacing missing values in numeric columns with mode

  repl_dict = df[columns_to_fix.index].mode().to_dict(orient='records')[0]
  df = df.fillna(repl_dict)

  #creating more features

  Remod_age = df['Yr Sold'] - df['Year Remod/Add']

  #Adding to datafreame

  df['Remod_age'] = Remod_age

  #dropping columns not useful/columns that leak information to model


  df = df.drop(["PID", "Order"], axis=1)

  df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

  return df


def select_features(df, target_corr=0.5, unique_thresh=7):
  num_df = df.select_dtypes(include = ['integer', 'float'])
  target_corr = num_df.corr()['SalePrice'].abs().sort_values(ascending = False)

  df = df.drop(target_corr[target_corr < 0.5].index, axis = 1)

  
  categ_features = ["Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
  "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
  "Misc Feature", "Sale Type", "Sale Condition"]

  #list of categorical columns
  transform_cols = []
  for i in categ_features:
      if i in transform_df.columns:
          transform_cols.append(i)

  #unique values in each 
  unique = df[transform_cols].apply(lambda i: len(i.value_counts())).sort_values(ascending = False)


  drop_notunique_cols = unique[unique > 7].index


  df = df.drop(drop_notunique_cols, axis=1)


# make rest of text columns to categorical
  text_columns = df.select_dtypes(include=['object'])
  for i in text_columns:
      df[i] = df[i].astype('category')
    
  df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category'])) ], axis=1)

  return df


def train_and_test(df, k=0):
  
  #splitting data into train and test 
  train = df[0:1460]
  test = df[1460:]
  
  #selecting numerical columns only
  numerical = numeric_df = df.select_dtypes(include=['integer', 'float'])
  
 
   #setting features and target 
  features = numeric_df.columns.drop('SalePrice')
  target = 'SalePrice'

  #initializing model
  lr = LinearRegression()

  if k == 0:
      train = df[:1460]
      test = df[1460:]

      #fitting model
      lr.fit(train[features], train[target])

      #making predictions
      predicted_y = lr.predict(test[features])
   
      #computing MSE
      mse = mean_squared_error(test[target],predicted_y)

      #computing RMSE
      rmse = np.sqrt(mse)
 
      return rmse

  
  if k == 1:
        
        shuf_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train[target])
        predictions_1 = lr.predict(test[features])        
        
        mse_1 = mean_squared_error(test[target], predictions_1)
        rmse_1 = np.sqrt(mse_1)
        
        lr.fit(test[features], test[target])
        predictions_2 = lr.predict(train[features])        
       
        mse_2 = mean_squared_error(train[target], predictions_2)
        rmse_2 = np.sqrt(mse_2)
        
        avg_rmse = np.mean([rmse_1, rmse_1])
        print(rmse_1)
        print(rmse_2)
        return avg_rmse
  else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train[target])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test[target], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse



In [60]:

# Reading in fresh dataframe
df = pd.read_csv("https://bit.ly/3boZCX4", sep='\t')
from sklearn.model_selection import KFold

# Using above functions to process dataframe and return RMSE
transform_data = transform_features(df)
filter_df = select_features(transform_data)
rmse = train_and_test(filter_df, k=5)

# Printing rmse from above
print(rmse)

# Iterating over k-values to analyze different results
for i in range(1,8):
    rmse = train_and_test(filtered_df, k=i)
    print(rmse)

[27810.76450980542, 33592.45682235849, 33821.115236579855, 31979.221146441694, 26448.34055839566]
30730.379654716224
36111.42877756493
28620.051410840868
36111.42877756493
[31479.20128122125, 31593.598738143428]
31536.400009682337
[27046.576961912593, 30145.212844766545, 37650.943244448805]
31614.244350375982
[32207.83020495364, 24072.28190370695, 36885.90540221181, 29707.939160221955]
30718.489167773587
[26852.008184652586, 30737.914515563232, 33752.801705825186, 30487.409805676372, 29671.27377281103]
30300.28159690568
[29036.181859280194, 25536.71327724581, 28065.935888257663, 47736.78220882887, 31103.959554583755, 26891.64699789822]
31395.203297682416
[36635.51970015881, 25557.979664649592, 31987.992873375497, 33294.483809121455, 24071.464177632068, 32076.12987918549, 26953.807869665787]
30082.482567684103


## 5. Next Steps

That's it for the guided steps. Here's some potenial next steps that you can take:

1. Continue iteration on feature engineering:
* Research some other approaches to feature engineering online around housing data.
* Visit the Kaggle kernels [page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels) page for this dataset to see approaches others took.

2. Improve your feature selection:
* Research ways of doing feature selection better with categorical columns (something we didn't cover in this particular course).